# Import Libraries

In [3]:
import csv, pprint, json, base64, zlib, struct

# Define Helper Functions

In [4]:
def base64ToJson(zippedString):
    json_str = zlib.decompress(base64.b64decode(zippedString)).decode()
    json_json = json.loads(json_str)
    return json_json

# Read In CSV File

In [5]:
!pwd

/home/alexchandra/Documents/Alex/capstone/adiConversion


In [6]:
!ls ./sampleFiles/4b80ff2eb1112815299d7a4e9a4a1957.csv

./sampleFiles/4b80ff2eb1112815299d7a4e9a4a1957.csv


In [7]:
!pwd

/home/alexchandra/Documents/Alex/capstone/adiConversion


In [8]:
csvFilename = "./sampleFiles/4b80ff2eb1112815299d7a4e9a4a1957.csv"

In [37]:
import csv

f = open(csvFilename)
csv_f = csv.reader(f)

# Will print the first row of the CSV - the first alarm for that admission
'''
for row in csv_f:
    #print(row)
    print(row[0]) # Alarm ID
    print(row[1]) # Time since Admission
    pprint.pprint(base64ToJson(row[2])) # Alarm data
    break    
'''
print('')

# Read In ADIBIN File

In [10]:
adibinFilename = "./sampleFiles/example.adibin"

Each File Header has a size of 68 bytes

Each Channel Header has a size of 96 bytes

The sample data for each channel follows its header.

#### [Excellent code snippets for working with Binary data in Python](https://www.devdungeon.com/content/working-binary-data-python "DevDungeon")

### Parse ADIBIN File

In [40]:
#Read in File Header and First Channel Title
with open(adibinFilename, "rb") as adibin_file:
    # Read the whole file at once
    #data = binary_file.read()
    #print(data)
    
    adibin_file.seek(0)  # Go to beginning
    fileHeader = adibin_file.read(68)
    channelTitle = adibin_file.read(96)
    print(fileHeader)
    print('\n')
    print(channelTitle)

b'CFWB\x01\x00\x00\x00\x11\x11\x11\x11\x11\x11q?\xe0\x07\x00\x00\n\x00\x00\x00\x1d\x00\x00\x00\t\x00\x00\x00/\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\n\x00\x00\x00!\x1c\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00'


b'I\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00mV\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x85\xebQ\xb8\x1e\x85\x03@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00'


In [17]:
len(fileHeader)

68

In [29]:
len(channelTitle)

96

### Define Format Strings for C Structs

#### [Relevant Python Documentation for Reading C Structs into Python](https://docs.python.org/3.5/library/struct.html "Python Docs for C Structs")

In [68]:
#Define Format Strings to decode Structs 

fileHeaderFormatString = "=4sldlllllddllll"
s1=struct.calcsize(fileHeaderFormatString)

channelTitleFormatString = "=32s32sdddd"
s2=struct.calcsize(channelTitleFormatString)

print(s1, s2)

68 96


In [69]:
tupleFileHeader = struct.unpack(fileHeaderFormatString, fileHeader)
tupleChannelHeader = struct.unpack(channelTitleFormatString, channelTitle)

In [70]:
print(tupleFileHeader)
print('\n')
print(tupleChannelHeader)

(b'CFWB', 1, 0.004166666666666667, 2016, 10, 29, 9, 47, 0.0, 0.0, 10, 7201, 0, 3)


(b'I\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', b'mV\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 2.44, 0.0, 1.0, 0.0)


In [73]:
Magic, Version, secsPerTick, Year, Month, Day, Hour, Minute, Second, trigger, NChannels, SamplesPerChannel,\
    TimeChannel, DataFormat = struct.unpack(fileHeaderFormatString, fileHeader)

In [76]:
ChannelTitle, Units, Scale, offset, RangeHigh, RangeLow = struct.unpack(channelTitleFormatString, channelTitle)
print(ChannelTitle)
print('\n')
print(ChannelTitle.decode('utf-8'))

b'I\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'


I                               


In [14]:
'''
Code snippet from ADIBinaryFormat.h that shows the structure of the FileHeader portion

* sizeof(char)   = 1 byte
* sizeof(short)  = 2 bytes
* sizeof(long)   = 4 bytes
* sizeof(double) = 8 bytes, i.e. 64 bit IEEE floating point
*

struct CFWBINARY
   {
   char     magic[4];            // always "CFWB"
   long     Version;             // = CFWB_VERSION
   double   secsPerTick;         // sampling interval in seconds

   // Trigger Date and time information
   long    Year;                // 4 digit year
   long    Month;               // months 1 - 12
   long    Day;                 // days 1 - 31
   long    Hour;                // hours 0 - 23
   long    Minute;              // minutes 0 - 59
   double   Second;              // seconds
   double   trigger;             // Amount of pretrigger data in seconds.

   long     NChannels;           // Number of channels
   long     SamplesPerChannel;   // Number of sample points per channel
   
   // The TimeChannel flag indicates that the sample time of each sample is
   // interleaved as the first column of data. This is only valid for the floating
   // point data formats. For all releases of TranslateBinary up to and including
   // v1.3, sample time data can be included but it is not used.
   long     TimeChannel;         // 1 = time included as first channel, 0 = not included
   long     DataFormat;          // 1 = double , 2 = float, 3 = 16-bit integer
   };
   
'''
print('')

In [51]:
with open(adibinFilename, "rb") as binary_file:
    data = binary_file.read()
    
    binary_file.seek(0)
    magic = binary_file.read(4)
    print(magic)

b'CFWB'


In [58]:
with open(adibinFilename, "rb") as binary_file:
    data = binary_file.read()
    
    binary_file.seek(0)
    magic = binary_file.read(4)
    version = binary_file.read(4)
    intVersion = int.from_bytes(version, byteorder='little')
    secsPerTick = binary_file.read(8)
    intSecsPerTick = int.from_bytes(secsPerTick, byteorder='little')
    print(magic, version, intVersion, secsPerTick, intSecsPerTick)
    

b'CFWB' b'\x01\x00\x00\x00' 1 b'\x11\x11\x11\x11\x11\x11q?' 4571453861756211473


In [42]:
import sys
#reload(sys)
#sys.setdefaultencoding('utf-8')
sys.getdefaultencoding()

'utf-8'

In [44]:
with open(adibinFilename, "rb") as binary_file:
    # Read the whole file at once
    data = binary_file.read()
    #print(data)
    
    binary_file.seek(0)  # Go to beginning
    couple_bytes = binary_file.read(68)
    print(couple_bytes.decode('utf-8'))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe0 in position 16: invalid continuation byte

In [45]:
binary_data = b'test'
text = binary_data.decode('utf-8')
print(text)

test


In [46]:
print(binary_data)

b'test'


In [47]:
base64_data = codecs.encode(binary_data, 'base64')

In [48]:
print(type(binary_message))

NameError: name 'binary_message' is not defined

In [49]:
print(type(base64_data))

<class 'bytes'>


In [50]:
print(base64_data)

b'dGVzdA==\n'
